In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

In [2]:
df = pd.read_csv('../../data/crawler/unified-events-statistics.csv')
def qtd_eventos(integer):
    return 45 - integer

qtd_eventos = list(map(qtd_eventos, df.isnull().sum(axis=1).tolist()))
df['qtd_eventos'] = qtd_eventos
df = df[df.qtd_eventos >= 30]
df = df.replace(['BLUE: first_blood',
    'BLUE: dragon',
    'BLUE: herald',
    'BLUE: first_tower_top',
    'BLUE: first_tower_mid',
    'BLUE: first_tower_bot',
    'BLUE: second_tower_top',
    'BLUE: second_tower_mid',
    'BLUE: second_tower_bot',
    'BLUE: third_tower_top',
    'BLUE: third_tower_mid',
    'BLUE: third_tower_bot',
    'BLUE: inhibitor_top',
    'BLUE: inhibitor_mid',
    'BLUE: inhibitor_bot',
    'BLUE: baron',
    'BLUE: elder_dragon',
    'BLUE: nexus_tower',
    'BLUE: nexus',
    'RED: first_blood',
    'RED: dragon',
    'RED: herald',
    'RED: first_tower_top',
    'RED: first_tower_mid',
    'RED: first_tower_bot',
    'RED: second_tower_top',
    'RED: second_tower_mid',
    'RED: second_tower_bot',
    'RED: third_tower_top',
    'RED: third_tower_mid',
    'RED: third_tower_bot',
    'RED: inhibitor_top',
    'RED: inhibitor_mid',
    'RED: inhibitor_bot',
    'RED: baron',
    'RED: elder_dragon',
    'RED: nexus_tower',
    'RED: nexus'], range(38))
df.head()

,golId,game,event1,event2,event3,event4,event5,event6,event7,event8,...,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,result,qtd_eventos
2,36866,ESPORTSTMNT02_2557390,19,2,20,24,2,4,20,3,...,0.0,20.0,0,0.0,0.00,4,1.0,2.8,0,31
7,36889,ESPORTSTMNT02_2557053,19,1,2,5,1,20,21,4,...,1.0,1.6,1,0.0,0.40,0,0.0,0.0,1,32
11,36894,ESPORTSTMNT02_2577994,0,20,2,1,5,3,6,20,...,0.0,0.0,1,0.0,12.00,0,0.0,0.0,1,30
33,36926,ESPORTSTMNT05_2530903,19,21,20,22,23,20,21,25,...,0.0,0.0,0,0.0,0.00,0,0.0,0.0,0,34
35,36928,ESPORTSTMNT05_2530937,0,20,21,1,22,3,24,2,...,0.0,0.0,2,0.0,1.75,0,0.0,0.0,1,35


In [3]:
y = df['result'].copy()
X = df.drop(['qtd_eventos','golId','result','game','event26','event31','event32','event33','event34','event35','event36','event37','event38','event39','event40','event41','event42','event43','event44','event45'],axis=1)
X.head()

,event1,event2,event3,event4,event5,event6,event7,event8,event9,event10,...,redJungleKDA,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA
2,19,2,20,24,2,4,20,3,6,1,...,0.00,1,0.0,20.0,0,0.0,0.00,4,1.0,2.8
7,19,1,2,5,1,20,21,4,1,3,...,0.00,1,1.0,1.6,1,0.0,0.40,0,0.0,0.0
11,0,20,2,1,5,3,6,20,2,4,...,13.67,0,0.0,0.0,1,0.0,12.00,0,0.0,0.0
33,19,21,20,22,23,20,21,25,5,24,...,0.00,0,0.0,0.0,0,0.0,0.00,0,0.0,0.0
35,0,20,21,1,22,3,24,2,4,7,...,0.00,0,0.0,0.0,2,0.0,1.75,0,0.0,0.0


In [4]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,stratify=y)
    scaler = StandardScaler()   
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test,scaler

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

executions = pd.DataFrame()

for i in range(50):
  print('execução número ' + str(i))
  X_train,X_test,y_train,y_test,scaler = preprocess_input(X,y)

  models = {
      'Logistic Regression': LogisticRegression(max_iter=10000),
      'Support Vector Machine (RBF Kernel)': SVC(C=1,gamma=0.01,kernel='sigmoid',max_iter=50000,probability=True),
      'Decission Tree': DecisionTreeClassifier(),
      'Adaboost': AdaBoostClassifier(),
      'Random Forest': RandomForestClassifier(),
      'Gradient Boosting Classifier': GradientBoostingClassifier(),
      'KNN': KNeighborsClassifier(n_neighbors=3),
      'Gaussian NB': GaussianNB()
  }

  for name, model in models.items():
    model.fit(X_train,y_train)

  scores_list = []

  for name,model in models.items():    
      scores_list.append({
      'model': name,
      'amount_events': '30',
      'execution': i,
      'Balanced Accuracy': balanced_accuracy_score(y_test,model.predict(X_test)),
      'Precision':  precision_score(y_test,model.predict(X_test)),
      'Recall': recall_score(y_test,model.predict(X_test)),
      'F1-Score': f1_score(y_test,model.predict(X_test)),
      'auc': roc_auc_score(y_test,model.predict_proba(X_test)[:,1])
      })
  # scores = pd.DataFrame(scores_list)
  executions = executions.append(scores_list)

execução número 0
execução número 1
execução número 2
execução número 3
execução número 4
execução número 5
execução número 6
execução número 7
execução número 8
execução número 9
execução número 10
execução número 11
execução número 12
execução número 13
execução número 14
execução número 15
execução número 16
execução número 17
execução número 18
execução número 19
execução número 20
execução número 21
execução número 22
execução número 23
execução número 24
execução número 25
execução número 26
execução número 27
execução número 28
execução número 29
execução número 30
execução número 31
execução número 32
execução número 33
execução número 34
execução número 35
execução número 36
execução número 37
execução número 38
execução número 39
execução número 40
execução número 41
execução número 42
execução número 43
execução número 44
execução número 45
execução número 46
execução número 47
execução número 48
execução número 49


In [6]:
avg_scores_list = []

for name, model in models.items():

    avg_balanced_acc = 0

    model_metrics = executions.loc[executions['model'] == name]
    avg_balanced_acc = model_metrics['Balanced Accuracy'].sum() / len(model_metrics['Balanced Accuracy'])
    avg_precision = model_metrics['Precision'].sum() / len(model_metrics['Precision'])
    avg_recall = model_metrics['Recall'].sum() / len(model_metrics['Recall'])
    avg_f_score = model_metrics['F1-Score'].sum() / len(model_metrics['F1-Score'])
    avg_auc = model_metrics['auc'].sum() / len(model_metrics['auc'])

    avg_scores_list.append({
      'model': name,
      'model_obj': model,
      'amount_events': '30',
      'Balanced Accuracy': avg_balanced_acc,
      'Precision': avg_precision,
      'Recall': avg_recall,
      'F1-Score': avg_f_score,
      'auc': avg_auc
      })
avg_scores = pd.DataFrame(avg_scores_list)

In [7]:
ordered_scores = avg_scores.sort_values(by='auc', ascending=False)

In [8]:
ordered_scores

,model,model_obj,amount_events,Balanced Accuracy,Precision,Recall,F1-Score,auc
4,Random Forest,"(DecisionTreeClassifier(max_features='auto', r...",30,0.896333,0.897175,0.876735,0.886078,0.951411
5,Gradient Boosting Classifier,([DecisionTreeRegressor(criterion='friedman_ms...,30,0.898073,0.894999,0.883265,0.888139,0.947672
1,Support Vector Machine (RBF Kernel),"SVC(C=1, gamma=0.01, kernel='sigmoid', max_ite...",30,0.888848,0.860576,0.900408,0.879250,0.940761
3,Adaboost,"(DecisionTreeClassifier(max_depth=1, random_st...",30,0.878461,0.865008,0.871837,0.866968,0.935732
7,Gaussian NB,GaussianNB(),30,0.881944,0.858758,0.886939,0.871389,0.935621
0,Logistic Regression,LogisticRegression(max_iter=10000),30,0.863352,0.848024,0.857551,0.851066,0.925949
6,KNN,KNeighborsClassifier(n_neighbors=3),30,0.825396,0.814772,0.803673,0.808094,0.886593
2,Decission Tree,DecisionTreeClassifier(),30,0.847354,0.830539,0.838776,0.833263,0.847354


In [9]:
# Ensemble Voting Classifier
from sklearn.ensemble import VotingClassifier

# Criando o ensemble com os classificadores
ensemble_classifier = VotingClassifier(estimators=[
    (ordered_scores.iloc[0]['model'], ordered_scores.iloc[0]['model_obj']),
    (ordered_scores.iloc[1]['model'], ordered_scores.iloc[1]['model_obj']),
    (ordered_scores.iloc[2]['model'], ordered_scores.iloc[2]['model_obj'])
], voting='soft')

ensemble_classifier = ensemble_classifier.fit(X_train, y_train)

In [10]:
ensemble_scores_list = []
ensemble_scores_list.append({
    'model': 'Ensemble Classifier',
    'model_obj': ensemble_classifier,
    'amount_events': 30,
    'Balanced Accuracy': balanced_accuracy_score(y_test,ensemble_classifier.predict(X_test)),
    'Precision':  precision_score(y_test,ensemble_classifier.predict(X_test)),
    'Recall': recall_score(y_test,ensemble_classifier.predict(X_test)),
    'F1-Score': f1_score(y_test,ensemble_classifier.predict(X_test)),
    'auc': roc_auc_score(y_test,ensemble_classifier.predict_proba(X_test)[:,1])
})

In [11]:
ensemble_scores = pd.DataFrame(ensemble_scores_list)
ensemble_scores_list

[{'model': 'Ensemble Classifier',
  'model_obj': VotingClassifier(estimators=[('Random Forest', RandomForestClassifier()),
                               ('Gradient Boosting Classifier',
                                GradientBoostingClassifier()),
                               ('Support Vector Machine (RBF Kernel)',
                                SVC(C=1, gamma=0.01, kernel='sigmoid',
                                    max_iter=50000, probability=True))],
                   voting='soft'),
  'amount_events': 30,
  'Balanced Accuracy': 0.9117952265652023,
  'Precision': 0.8545454545454545,
  'Recall': 0.9591836734693877,
  'F1-Score': 0.9038461538461537,
  'auc': 0.9612590799031477}]

In [12]:
header = ['model', 'amount_events', 'auc']
ensemble_scores.to_csv('../../data/models-analysis/models.csv', columns = header, mode='a',index=False,header=False)

In [13]:
header = ['model', 'amount_events', 'auc']
avg_scores.to_csv('../../data/models-analysis/models.csv', columns = header, mode='a',index=False,header=False)

In [14]:
models = pd.concat([ensemble_scores,avg_scores])
ordered_models = models.sort_values(by='auc', ascending=False)
best_model = ordered_models.iloc[0]['model_obj']
best_model

VotingClassifier(estimators=[('Random Forest', RandomForestClassifier()),
                             ('Gradient Boosting Classifier',
                              GradientBoostingClassifier()),
                             ('Support Vector Machine (RBF Kernel)',
                              SVC(C=1, gamma=0.01, kernel='sigmoid',
                                  max_iter=50000, probability=True))],
                 voting='soft')

In [23]:
import lime
import lime.lime_tabular

explainer = lime.lime_tabular.LimeTabularExplainer(X_train, class_names=['red win', 'blue win'], discretize_continuous=True)

In [16]:
exp = explainer.explain_instance(X_test[i], predict_fn, num_features=5)
exp.show_in_notebook()

NameError: name 'explainer' is not defined

In [ ]:
import pickle

with open("../../models/model-30.pkl", "wb") as f:
    pickle.dump(best_model, f)

with open("../../scalers/scaler-30.pkl", "wb") as f:
    pickle.dump(scaler, f)

with open("../../explainers/explainer-30.pkl", "wb") as f:
    pickle.dump(explainer, f)